# Pretrained BERT의 [CLS] token을 이용한 챗봇

1. 사용자의 질문(query)를 입력 받는다.
2. query를 pretrained BERT의 입력으로 넣어, query 문장에 해당하는 [CLS] token hidden을 얻는다.
3. 사전에 준비된 질의응답 DataSet에 존재하는 모든 질문들을 pretrained BERT의 입력으로 넣어 질문에 해당하는 [CLS] token hidden을 얻는다
4. query의 [CLS] token hidden과 질문들의 [CLS] token hidden간의 코사인 유사도를 구한다.
5. 가장 높은 코사인 유사도를 가진 질문의 답변을 변환시켜준다.
6. 위 과정을 반복한다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/자연어처리/ChatbotData.csv')
print(dataset.head())

                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


In [ ]:
chatbot_Q = dataset['Q']
chatbot_A = dataset['A']

print(chatbot_Q[:])
print(chatbot_A[:])

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object
0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object


In [ ]:
# !pip install transformers

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

model_name = 'bert-base-multilingual-cased'
# BertTokenizerFaset class

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.parameters

<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): BertOutput(
          (dense): Linear(in_features=3072, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (pooler): BertPooler(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
  )
)>

query [CLS] token hidden 확인

# [CLS] tokens을 얻기 위한 함수

In [ ]:
def get_cls_token(sent_a):
  model.eval()
  tokenized_sent = tokenizer(
      sent_a,
      return_tensors='pt',
      truncation=True,
      add_special_tokens=True,
      max_length=128
  )
  with torch.no_grad(): # 그래디언트 계산 비활성화
    outputs = model(
        input_ids = tokenized_sent['input_ids'],
        attention_mask = tokenized_sent['attention_mask'],
        token_type_ids = tokenized_sent['token_type_ids']
    )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [ ]:
# query [CLS'] token hidden 확인

In [ ]:
query = '12시 땡!'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden.shape)
print(query_cls_hidden)

(1, 768)
[[-9.83642191e-02 -2.87351366e-02 -6.73831284e-01  3.49760890e-01
  -1.38188735e-01  1.05787061e-01 -1.10930532e-01 -2.34378964e-01
   3.44763428e-01  1.00279570e+00  7.52500892e-01  2.67197222e-01
   7.43559897e-01  1.53217241e-01 -4.97159630e-01  2.74949521e-01
  -3.50510888e-02  7.44396374e-02  3.55907887e-01 -3.20007741e-01
   3.57761353e-01 -2.35026658e-01 -5.35087228e-01  2.12714076e-01
  -2.50891089e-01  2.92959958e-01  2.72913247e-01  3.10162872e-01
   2.07069665e-01  4.88869458e-01  6.50326371e-01  1.76527813e-01
  -1.51340485e-01  3.69304121e-01  1.69007450e-01 -3.13699156e-01
  -1.28002059e+00 -2.44472593e-01 -1.27538264e-01  7.67993182e-02
  -2.84482818e-02  9.28857997e-02 -4.91078049e-01  1.65591553e-01
  -2.61630677e-02  9.91972566e-01  3.23325545e-01  9.53709856e-02
   3.93901706e-01 -8.85689139e-01  1.24107905e-01 -4.73038286e-01
   1.27066076e-01 -6.43099785e-01  4.89597917e-01  5.00157654e-01
   2.58109599e-01 -1.90400198e-01  2.73511231e-01 -4.32527393e-01
 

chatbot 데이터셋의 질문 [CLS] token hidden 확인

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dataset_cls_hidden = []
for q in chatbot_Q:
  q_cls = get_cls_token(q)
  dataset_cls_hidden.append(q_cls)

dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)

print(dataset_cls_hidden.shape)
print(dataset_cls_hidden)

(11823, 768)
[[-9.83642191e-02 -2.87351366e-02 -6.73831284e-01 ...  7.08658695e-01
   6.99949712e-02  3.66942465e-01]
 [ 6.49076402e-02  7.18163103e-02 -1.40971839e-01 ...  1.38332799e-01
   1.65951878e-01  1.12920955e-01]
 [ 4.07720829e-04  2.28372309e-02  2.58352071e-01 ...  1.35420769e-01
   2.67340571e-01  9.57453027e-02]
 ...
 [ 1.53779060e-01 -2.82412823e-02 -8.10718387e-02 ...  2.28785351e-01
   1.65325869e-02  1.29616097e-01]
 [-4.51040231e-02  1.04225680e-01 -1.89268276e-01 ...  5.91341257e-01
   2.69855797e-01 -2.99956594e-02]
 [ 3.52816507e-02  3.98816215e-03  1.61026374e-01 ...  6.02968156e-01
   4.56987461e-03  4.15169686e-01]]


# 코사인 유사도

In [ ]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)
print(cos_sim)

[[1.         0.50382656 0.49437356 ... 0.45996094 0.53037757 0.52563286]]


In [ ]:
# chatbot 데이터 셋 중 가장 유사도가 높은 질문 선택 및 답변
top_question = np.argmax(cos_sim)

print('my q:', query)
print('A', chatbot_A[top_question])

my q: 12시 땡!
A 하루가 또 가네요.
